In [1]:
# ## consume to mongodb
# from kafka import KafkaConsumer
# import sys, json, pymongo
# # from elasticsearch import Elasticsearch
# # import time, datetime


# if __name__ == "__main__":

#     # 與 MongoDB連線
#     client = pymongo.MongoClient(host="mongodb", port=27017)
#     # 指定為 test 資料庫
#     db = client.test
#     # 指定 temp_humidity 集合, MongoDB的每個資料庫又包含許多集合(collection), 類似於關聯性資料庫中的表
#     collection = db.bpm

#     # 設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
#     consumer = KafkaConsumer(
#         # 指定Kafka集群伺服器
#         bootstrap_servers=["kafka:9092"],
#         # ConsumerGroup的名稱, 可以不指定
#         #group_id="cg_001",
#         # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
#         # key_deserializer=bytes.decode,
#         # 指定msgValue的反序列化器
#         #value_deserializer=bytes.decode,
#         value_deserializer=lambda m: json.loads(m.decode('ascii')),
#         # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
#         auto_offset_reset="earliest",
#     )
   
#     # 讓Consumer向Kafka集群訂閱指定的topic
#     consumer.subscribe(topics="bpm")
    
#     #es = Elasticsearch('http://elasticsearch:9200')
    
    
#     # 持續的拉取Kafka有進來的訊息
#     try:
#         print("Now listening for incoming messages ...")
#         # 持續監控是否有新的record進來
#         for record in consumer:
#             topic = record.topic
#             partition = record.partition
#             offset = record.offset
#             timestamp = record.timestamp
#             # 取出msgKey與msgValue
#             msgKey = record.key
#             msgValue = record.value
#             # 秀出metadata與msgKey & msgValue訊息
#             print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition, 
#                                                                               record.offset, record.key, record.value))
#             j = {"device_id": msgValue["device_id"], "timestamp": msgValue["timestamp"], "bpm": msgValue["bpm"]}
#             print("json=",  j)
            
#             # 將資料存入 mongodb
#             # 存入單筆
#             result = collection.insert_one(j)
#             # 存入多筆
#             #result = collection.insert_many()
#             print(result)
#             #res = es.index(index="temp_humidity1", doc_type='tweet', id=datetime.now(), body=j)
#             #print(res['result'])

#     except:
#         # 錯誤處理
#         e_type, e_value, e_traceback = sys.exc_info()
#         print("type ==> %s" % (e_type))
#         print("value ==> %s" % (e_value))
#         print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
#         print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
#         print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
#     finally:
#         consumer.close()

In [ ]:
## consume to Elasticsearch
from kafka import KafkaConsumer
import sys, json
# import pymongo
from elasticsearch import Elasticsearch
import time, datetime


if __name__ == "__main__":

    # 與 MongoDB連線
#     client = pymongo.MongoClient(host="mongodb", port=27017)
#     # 指定為 test 資料庫
#     db = client.test
#     # 指定 temp_humidity 集合, MongoDB的每個資料庫又包含許多集合(collection), 類似於關聯性資料庫中的表
#     collection = db.bpm

    # 設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # 指定Kafka集群伺服器
        bootstrap_servers=["kafka:9092"],
        # ConsumerGroup的名稱, 可以不指定
        #group_id="cg_001",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
        # key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        #value_deserializer=bytes.decode,
        value_deserializer=lambda m: json.loads(m.decode('ascii')),
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="earliest",
    )
   
    # 讓Consumer向Kafka集群訂閱指定的topic
    consumer.subscribe(topics="bpm")
    
    es = Elasticsearch('http://elasticsearch:9200')
    
    
    # 持續的拉取Kafka有進來的訊息
    try:
        print("Now listening for incoming messages ...")
        # 持續監控是否有新的record進來
        for record in consumer:
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            # 秀出metadata與msgKey & msgValue訊息
            time = datetime.datetime.now()
            print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition, 
                                                                              record.offset, record.key, record.value))
            j = {"device_id": msgValue["device_id"], "timestamp": time, "bpm": int(msgValue["bpm"])}
            print("json=",  j)
            
#             # 將資料存入 mongodb
#             # 存入單筆
#             result = collection.insert_one(j)
#             # 存入多筆
#             # result = collection.insert_many()
#             print(result)


            res = es.index(index="bpm", doc_type='tweet', id=time, body=j)
            print(res['result'])

    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        consumer.close()

Now listening for incoming messages ...
topic=bpm, partition=0, offset=370 : (key=None, value={'timestamp': '2019-10-17 11:41:00', 'bpm': '53', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 291901), 'bpm': 53}
created
topic=bpm, partition=0, offset=371 : (key=None, value={'timestamp': '2019-10-17 11:41:06', 'bpm': '135', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 298522), 'bpm': 135}
created
topic=bpm, partition=0, offset=372 : (key=None, value={'timestamp': '2019-10-17 11:41:08', 'bpm': '103', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 303720), 'bpm': 103}
created
topic=bpm, partition=0, offset=373 : (key=None, value={'timestamp': '2019-10-17 11:41:09', 'bpm': '84', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 308382), 'bpm': 84}
created
topic=bp

created
topic=bpm, partition=0, offset=417 : (key=None, value={'timestamp': '2019-10-17 11:42:55', 'bpm': '32', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 473697), 'bpm': 32}
created
topic=bpm, partition=0, offset=418 : (key=None, value={'timestamp': '2019-10-17 11:42:57', 'bpm': '32', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 477413), 'bpm': 32}
created
topic=bpm, partition=0, offset=419 : (key=None, value={'timestamp': '2019-10-17 11:42:59', 'bpm': '32', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 480758), 'bpm': 32}
created
topic=bpm, partition=0, offset=420 : (key=None, value={'timestamp': '2019-10-17 11:43:01', 'bpm': '31', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 484259), 'bpm': 31}
created
topic=bpm, partition=0, offset=421 : (key=No

created
topic=bpm, partition=0, offset=470 : (key=None, value={'timestamp': '2019-10-17 11:45:08', 'bpm': '63', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 676476), 'bpm': 63}
created
topic=bpm, partition=0, offset=471 : (key=None, value={'timestamp': '2019-10-17 11:45:10', 'bpm': '57', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 681410), 'bpm': 57}
created
topic=bpm, partition=0, offset=472 : (key=None, value={'timestamp': '2019-10-17 11:45:11', 'bpm': '52', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 684685), 'bpm': 52}
created
topic=bpm, partition=0, offset=473 : (key=None, value={'timestamp': '2019-10-17 11:45:13', 'bpm': '48', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 688742), 'bpm': 48}
created
topic=bpm, partition=0, offset=474 : (key=No

created
topic=bpm, partition=0, offset=527 : (key=None, value={'timestamp': '2019-10-17 11:48:16', 'bpm': '33', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 879868), 'bpm': 33}
created
topic=bpm, partition=0, offset=528 : (key=None, value={'timestamp': '2019-10-17 11:48:18', 'bpm': '33', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 884790), 'bpm': 33}
created
topic=bpm, partition=0, offset=529 : (key=None, value={'timestamp': '2019-10-17 11:48:23', 'bpm': '52', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 888473), 'bpm': 52}
created
topic=bpm, partition=0, offset=530 : (key=None, value={'timestamp': '2019-10-17 11:48:25', 'bpm': '49', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 0, 893141), 'bpm': 49}
created
topic=bpm, partition=0, offset=531 : (key=No

created
topic=bpm, partition=0, offset=585 : (key=None, value={'timestamp': '2019-10-17 11:53:26', 'bpm': '65', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 81713), 'bpm': 65}
created
topic=bpm, partition=0, offset=586 : (key=None, value={'timestamp': '2019-10-17 11:53:38', 'bpm': '47', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 86405), 'bpm': 47}
created
topic=bpm, partition=0, offset=587 : (key=None, value={'timestamp': '2019-10-17 11:53:42', 'bpm': '73', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 90378), 'bpm': 73}
created
topic=bpm, partition=0, offset=588 : (key=None, value={'timestamp': '2019-10-17 11:53:44', 'bpm': '65', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 94890), 'bpm': 65}
created
topic=bpm, partition=0, offset=589 : (key=None, 

created
topic=bpm, partition=0, offset=643 : (key=None, value={'timestamp': '2019-10-17 11:57:58', 'bpm': '67', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 287247), 'bpm': 67}
created
topic=bpm, partition=0, offset=644 : (key=None, value={'timestamp': '2019-10-17 11:58:17', 'bpm': '46', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 290665), 'bpm': 46}
created
topic=bpm, partition=0, offset=645 : (key=None, value={'timestamp': '2019-10-17 11:58:20', 'bpm': '105', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 294424), 'bpm': 105}
created
topic=bpm, partition=0, offset=646 : (key=None, value={'timestamp': '2019-10-17 11:58:29', 'bpm': '229', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 298677), 'bpm': 229}
created
topic=bpm, partition=0, offset=647 : (ke

created
topic=bpm, partition=0, offset=698 : (key=None, value={'timestamp': '2019-10-17 12:04:09', 'bpm': '49', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 490862), 'bpm': 49}
created
topic=bpm, partition=0, offset=699 : (key=None, value={'timestamp': '2019-10-17 12:04:14', 'bpm': '67', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 494922), 'bpm': 67}
created
topic=bpm, partition=0, offset=700 : (key=None, value={'timestamp': '2019-10-17 12:04:17', 'bpm': '57', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 498978), 'bpm': 57}
created
topic=bpm, partition=0, offset=701 : (key=None, value={'timestamp': '2019-10-17 12:04:18', 'bpm': '53', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 502935), 'bpm': 53}
created
topic=bpm, partition=0, offset=702 : (key=No

created
topic=bpm, partition=0, offset=754 : (key=None, value={'timestamp': '2019-10-17 12:09:27', 'bpm': '134', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 693449), 'bpm': 134}
created
topic=bpm, partition=0, offset=755 : (key=None, value={'timestamp': '2019-10-17 12:09:30', 'bpm': '113', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 697189), 'bpm': 113}
created
topic=bpm, partition=0, offset=756 : (key=None, value={'timestamp': '2019-10-17 12:09:36', 'bpm': '191', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 701317), 'bpm': 191}
created
topic=bpm, partition=0, offset=757 : (key=None, value={'timestamp': '2019-10-17 12:09:43', 'bpm': '87', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 1, 705084), 'bpm': 87}
created
topic=bpm, partition=0, offset=758 : (

topic=bpm, partition=0, offset=791 : (key=None, value={'timestamp': '2019-10-23 12:48:43', 'bpm': '95', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 45, 158533), 'bpm': 95}
created
topic=bpm, partition=0, offset=792 : (key=None, value={'timestamp': '2019-10-23 12:48:47', 'bpm': '157', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 48, 48, 813164), 'bpm': 157}
created
topic=bpm, partition=0, offset=793 : (key=None, value={'timestamp': '2019-10-23 12:48:53', 'bpm': '39', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 49, 0, 394827), 'bpm': 39}
created
topic=bpm, partition=0, offset=794 : (key=None, value={'timestamp': '2019-10-23 12:49:02', 'bpm': '104', 'device_id': 'yoyo'})
json= {'device_id': 'yoyo', 'timestamp': datetime.datetime(2019, 10, 23, 4, 49, 3, 761478), 'bpm': 104}
created
topic=bpm, partition=0, offset=795 : (key=None

In [ ]:
print(datetime.datetime.now())